In [1]:

!pip install openai openfabric_pysdk torch transformers accelerate sentence_transformers gradio huggingface_hub

ModuleNotFoundError: No module named 'marshmallow.base'

In [ ]:

import os
import sys
import json
import time
import datetime
import sqlite3
import base64
from pathlib import Path
import requests
from io import BytesIO
from PIL import Image
import torch
from huggingface_hub import InferenceClient
from openfabric_pysdk.context import OpenfabricExecutionRateLimiter
from openfabric_pysdk.loader import ConfigClass
from openfabric_pysdk.transport import OpenFabricStub

# Check if GPU is available
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# Create directories for storage
os.makedirs("outputs/images", exist_ok=True)
os.makedirs("outputs/models", exist_ok=True)
os.makedirs("memory", exist_ok=True)

# Setup memory database
conn = sqlite3.connect("memory/creative_memory.db")
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS creations (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp TEXT,
    original_prompt TEXT,
    enhanced_prompt TEXT,
    image_path TEXT,
    model_path TEXT,
    tags TEXT
)
''')
conn.commit()

# HuggingFace LLM Configuration
class HuggingFaceLLM:
    def __init__(self):
        print("Setting up HuggingFace Inference Client...")
        # Get HuggingFace API token from environment or prompt
        self.api_token = os.environ.get("HF_TOKEN")
        if not self.api_token:
            print("Please set your HF_TOKEN environment variable")
            self.api_token = input("Enter your HuggingFace API token: ")
            os.environ["HF_TOKEN"] = self.api_token

        # Initialize the inference client
        self.client = InferenceClient(token=self.api_token)

        # Choose a model - default to Meta's Llama 3 8B Instruct model
        self.model_name = "meta-llama/Llama-3-8b-instruct"
        print(f"HuggingFace Inference Client set up with model: {self.model_name}")

    def enhance_prompt(self, user_prompt):
        """Enhance the user prompt for better image generation using HuggingFace's hosted model"""
        system_message = """
        You are a creative AI assistant specialized in enhancing prompts for image generation.
        Your job is to take a simple prompt and expand it with vivid details, artistic style,
        lighting, mood, and composition. Keep the core idea intact but make it visually rich.
        """

        # Format prompt for llama-3 format
        prompt = f"""<|system|>
{system_message}
<|user|>
I want to generate an image of: {user_prompt}
Please enhance this prompt to make it more detailed and visually appealing.
<|assistant|>
"""

        try:
            # Use the HuggingFace inference API
            response = self.client.text_generation(
                prompt,
                model=self.model_name,
                max_new_tokens=256,
                temperature=0.7,
                do_sample=True
            )

            enhanced_prompt = response.strip()

            print(f"Original prompt: {user_prompt}")
            print(f"Enhanced prompt: {enhanced_prompt}")

            return enhanced_prompt

        except Exception as e:
            print(f"Error with HuggingFace inference: {e}")
            # Fallback to original prompt if enhancement fails
            print("Falling back to original prompt")
            return user_prompt

# Openfabric SDK Configuration
class OpenfabricConnector:
    def __init__(self):
        # You would need to set your Openfabric API key
        self.api_key = os.environ.get("OPENFABRIC_API_KEY")
        if not self.api_key:
            print("Please set your OPENFABRIC_API_KEY environment variable")
            self.api_key = input("Enter your Openfabric API key: ")
            os.environ["OPENFABRIC_API_KEY"] = self.api_key

        # App IDs for the services
        self.text2image_app_id = "f0997a01-d6d3-a5fe-53d8-561300318557"
        self.image2model_app_id = "69543f29-4d41-4afc-7f29-3d51591f11eb"

        # Initialize the stubs
        self.text2image_stub = self._init_stub(self.text2image_app_id)
        self.image2model_stub = self._init_stub(self.image2model_app_id)

    def _init_stub(self, app_id):
        """Initialize and return an Openfabric stub for the specified app"""
        config = ConfigClass()
        config.manifest_url = f"https://api.openfabric.network/v1/app/{app_id}/manifest"
        config.schema_url = f"https://api.openfabric.network/v1/app/{app_id}/schema"

        print(f"Initializing stub for app ID: {app_id}")
        try:
            stub = OpenFabricStub(config)
            print(f"Stub initialized successfully for {app_id}")
            return stub
        except Exception as e:
            print(f"Error initializing stub: {e}")
            return None

    def generate_image(self, prompt):
        """Generate an image from text using the Text-to-Image app"""
        print(f"Generating image from prompt: {prompt}")

        try:
            # Create the request payload
            request = {
                "prompt": prompt,
                "negative_prompt": "blurry, low quality, distorted, deformed",
                "width": 768,
                "height": 768,
                "num_inference_steps": 50,
                "guidance_scale": 7.5
            }

            # Execute the request with rate limiting
            with OpenfabricExecutionRateLimiter():
                response = self.text2image_stub.execute(request)

            if not response or "image" not in response:
                print("Failed to generate image: No valid response")
                return None

            # Decode the base64 image
            image_data = base64.b64decode(response["image"])
            image = Image.open(BytesIO(image_data))

            # Save the image
            timestamp = int(time.time())
            image_path = f"outputs/images/generated_{timestamp}.png"
            image.save(image_path)
            print(f"Image saved to {image_path}")

            return {
                "image": image,
                "path": image_path
            }

        except Exception as e:
            print(f"Error generating image: {e}")
            return None

    def generate_3d_model(self, image_data):
        """Generate a 3D model from an image using the Image-to-3D app"""
        print("Converting image to 3D model...")

        try:
            # Prepare the image for the request
            buffered = BytesIO()
            image_data["image"].save(buffered, format="PNG")
            image_base64 = base64.b64encode(buffered.getvalue()).decode("utf-8")

            # Create the request payload
            request = {
                "image": image_base64
            }

            # Execute the request with rate limiting
            with OpenfabricExecutionRateLimiter():
                response = self.image2model_stub.execute(request)

            if not response or "model" not in response:
                print("Failed to generate 3D model: No valid response")
                return None

            # Decode and save the 3D model
            timestamp = int(time.time())
            model_data = base64.b64decode(response["model"])
            model_path = f"outputs/models/model_{timestamp}.glb"

            with open(model_path, "wb") as f:
                f.write(model_data)

            print(f"3D model saved to {model_path}")

            return {
                "model_path": model_path
            }

        except Exception as e:
            print(f"Error generating 3D model: {e}")
            return None

class MemoryManager:
    def __init__(self, db_path="memory/creative_memory.db"):
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()

    def save_creation(self, original_prompt, enhanced_prompt, image_path, model_path, tags=None):
        """Save a creation to the memory database"""
        if tags is None:
            tags = []

        # Extract keywords from the prompt for automatic tagging
        if not tags:
            # Simple keyword extraction
            keywords = [word for word in enhanced_prompt.lower().split()
                     if len(word) > 3 and word not in ["with", "and", "the", "that", "this"]]
            tags = keywords[:5]  # Take top 5 keywords

        timestamp = datetime.datetime.now().isoformat()

        self.cursor.execute(
            "INSERT INTO creations (timestamp, original_prompt, enhanced_prompt, image_path, model_path, tags) VALUES (?, ?, ?, ?, ?, ?)",
            (timestamp, original_prompt, enhanced_prompt, image_path, model_path, ",".join(tags))
        )
        self.conn.commit()

        return self.cursor.lastrowid

    def search_creations(self, query=None, limit=5):
        """Search for creations in the memory database"""
        if query:
            # Search in prompts and tags
            self.cursor.execute(
                "SELECT * FROM creations WHERE original_prompt LIKE ? OR enhanced_prompt LIKE ? OR tags LIKE ? ORDER BY timestamp DESC LIMIT ?",
                (f"%{query}%", f"%{query}%", f"%{query}%", limit)
            )
        else:
            # Get recent creations
            self.cursor.execute("SELECT * FROM creations ORDER BY timestamp DESC LIMIT ?", (limit,))

        return self.cursor.fetchall()

    def get_creation_by_id(self, creation_id):
        """Get a specific creation by ID"""
        self.cursor.execute("SELECT * FROM creations WHERE id = ?", (creation_id,))
        return self.cursor.fetchone()

# Main creative pipeline
class CreativePipeline:
    def __init__(self):
        self.llm = HuggingFaceLLM()  # Now using HuggingFace instead of local LLM
        self.openfabric = OpenfabricConnector()
        self.memory = MemoryManager()

    def create_from_prompt(self, user_prompt, enhance=True):
        """Full pipeline: prompt → enhanced prompt → image → 3D model"""
        # Step 1: Enhance the prompt with the LLM
        if enhance:
            enhanced_prompt = self.llm.enhance_prompt(user_prompt)
        else:
            enhanced_prompt = user_prompt

        # Step 2: Generate image from enhanced prompt
        image_data = self.openfabric.generate_image(enhanced_prompt)
        if not image_data:
            return {"error": "Failed to generate image"}

        # Step 3: Generate 3D model from image
        model_data = self.openfabric.generate_3d_model(image_data)
        if not model_data:
            return {"error": "Failed to generate 3D model"}

        # Step 4: Save to memory
        creation_id = self.memory.save_creation(
            original_prompt=user_prompt,
            enhanced_prompt=enhanced_prompt,
            image_path=image_data["path"],
            model_path=model_data["model_path"]
        )

        return {
            "creation_id": creation_id,
            "original_prompt": user_prompt,
            "enhanced_prompt": enhanced_prompt,
            "image_path": image_data["path"],
            "model_path": model_data["model_path"]
        }

    def search_memory(self, query=None, limit=5):
        """Search memories for previous creations"""
        return self.memory.search_creations(query, limit)

# Set up a simple Gradio UI
import gradio as gr

def create_gradio_interface(pipeline):
    def generate(prompt):
        result = pipeline.create_from_prompt(prompt)
        if "error" in result:
            return None, None, result["error"]

        # Display the image and provide a download link for the 3D model
        image = Image.open(result["image_path"])
        model_path = result["model_path"]

        return image, model_path, f"""
        **Original Prompt:** {result['original_prompt']}

        **Enhanced Prompt:** {result['enhanced_prompt']}

        **Creation ID:** {result['creation_id']}

        The 3D model is available for download.
        """

    def search(query):
        results = pipeline.search_memory(query)
        if not results:
            return "No creations found matching your query."

        output = "## Found Creations\n\n"
        for r in results:
            creation_id, timestamp, original, enhanced, img_path, model_path, tags = r
            timestamp_dt = datetime.datetime.fromisoformat(timestamp)
            formatted_time = timestamp_dt.strftime("%Y-%m-%d %H:%M:%S")

            output += f"### ID: {creation_id} - {formatted_time}\n"
            output += f"**Original:** {original}\n"
            output += f"**Enhanced:** {enhanced}\n"
            output += f"**Tags:** {tags}\n"
            output += f"**Image:** {img_path}\n"
            output += f"**Model:** {model_path}\n\n"

        return output

    with gr.Blocks() as demo:
        gr.Markdown("# Creative Partner: Text → Image → 3D Pipeline")

        with gr.Tab("Create"):
            with gr.Row():
                prompt_input = gr.Textbox(label="Enter your creative prompt", lines=3, placeholder="A glowing dragon standing on a cliff at sunset")
                generate_btn = gr.Button("Generate")

            with gr.Row():
                image_output = gr.Image(label="Generated Image")
                model_output = gr.File(label="3D Model (GLB format)")

            result_text = gr.Markdown()

            generate_btn.click(
                generate,
                inputs=[prompt_input],
                outputs=[image_output, model_output, result_text]
            )

        with gr.Tab("Memory"):
            with gr.Row():
                search_input = gr.Textbox(label="Search your creations", placeholder="dragon, sunset, cyberpunk...")
                search_btn = gr.Button("Search")

            memory_results = gr.Markdown()

            search_btn.click(
                search,
                inputs=[search_input],
                outputs=[memory_results]
            )

    return demo

# Add functionality to select different HuggingFace models
def select_model(model_name):
    global pipeline
    pipeline.llm.model_name = model_name
    return f"Model changed to: {model_name}"

# Main execution
def main():
    print("Initializing Creative Pipeline...")
    pipeline = CreativePipeline()

    # Example 1: Direct use of the pipeline
    print("\n=== Example: Generating a dragon on a cliff ===")
    result = pipeline.create_from_prompt("A glowing dragon standing on a cliff at sunset")
    print(f"Creation result: {json.dumps(result, indent=2)}")

    # Launch Gradio interface
    print("\n=== Launching Gradio Interface ===")
    demo = create_gradio_interface(pipeline)

    # Add model selection dropdown
    with demo:
        with gr.Tab("Settings"):
            models = [
                "meta-llama/Llama-3-8b-instruct",
                "mistralai/Mistral-7B-Instruct-v0.2",
                "google/gemma-7b-it",
                "teknium/OpenHermes-2.5-Mistral-7B"
            ]
            model_dropdown = gr.Dropdown(
                choices=models,
                value="meta-llama/Llama-3-8b-instruct",
                label="Select HuggingFace Model"
            )
            model_output = gr.Markdown()
            model_dropdown.change(select_model, inputs=[model_dropdown], outputs=[model_output])

    demo.launch(share=True)

if __name__ == "__main__":
    # Set API keys from environment if available
    if os.environ.get("OPENFABRIC_API_KEY") is None:
        os.environ["OPENFABRIC_API_KEY"] = input("Enter your Openfabric API key: ")

    if os.environ.get("HF_TOKEN") is None:
        os.environ["HF_TOKEN"] = input("")

    main()

In [ ]:
!pip install